In [2]:
import torch
import torch.nn as nn
import string

# Define the RNN model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        output, hidden = self.rnn(x, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size).to(device)

# Function to convert character to tensor and vice versa
def char_to_tensor(char):
    all_letters = string.ascii_letters + " .,;'"
    return torch.tensor([all_letters.find(char)], dtype=torch.long).to(device)

def tensor_to_char(output):
    all_letters = string.ascii_letters + " .,;'"
    _, topi = output.topk(1)
    return all_letters[topi.item()]

# Define training parameters
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
input_size = len(string.ascii_letters + " .,;'")
hidden_size = 128
output_size = input_size
seq_len = 100
num_epochs = 1000
learning_rate = 0.001

# Initialize the model and move it to the device
model = CharRNN(input_size, hidden_size, output_size).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training data
text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
text_len = len(text)

# Training loop
for epoch in range(num_epochs):
    for i in range(0, text_len - seq_len, seq_len):
        inputs = torch.cat([char_to_tensor(char) for char in text[i:i+seq_len]]).view(1, -1)
        targets = torch.cat([char_to_tensor(char) for char in text[i+1:i+seq_len+1]]).view(1, -1)

        optimizer.zero_grad()
        hidden = model.init_hidden(1)

        loss = 0
        for j in range(seq_len):
            output, hidden = model(inputs[:,j].unsqueeze(0), hidden)
            loss += criterion(output.view(1, -1), targets[:,j])

        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

# Example usage for text generation
start_char = 'L'
num_predict = 100

with torch.no_grad():
    input_char = start_char
    hidden = model.init_hidden(1)

    for _ in range(num_predict):
        input_tensor = char_to_tensor(input_char)
        output, hidden = model(input_tensor.unsqueeze(0), hidden)
        predicted_char = tensor_to_char(output.squeeze(0))
        print(predicted_char, end='')
        input_char = predicted_char


Epoch [0/1000], Loss: 391.9741
Epoch [100/1000], Loss: 29.8055
Epoch [200/1000], Loss: 13.3521
Epoch [300/1000], Loss: 8.5025
Epoch [400/1000], Loss: 6.2372
Epoch [500/1000], Loss: 4.9217
Epoch [600/1000], Loss: 4.0907
Epoch [700/1000], Loss: 3.4471
Epoch [800/1000], Loss: 2.9678
Epoch [900/1000], Loss: 2.5898
orem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore e

In [3]:
import torch
import torch.nn as nn
import string

# Define the RNN model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        output, hidden = self.rnn(x, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size).to(device)

# Define training parameters
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
input_size = len(string.ascii_letters + " .,;'")
hidden_size = 128
output_size = input_size
seq_len = 100
num_epochs = 1000
learning_rate = 0.001

# Initialize the model and move it to the device
model = CharRNN(input_size, hidden_size, output_size).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training data
text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
text_len = len(text)

# Convert characters to indices and vice versa
all_letters = string.ascii_letters + " .,;'"
char_to_index = {char: i for i, char in enumerate(all_letters)}
index_to_char = {i: char for i, char in enumerate(all_letters)}

# Training loop
for epoch in range(num_epochs):
    for i in range(0, text_len - seq_len, seq_len):
        inputs = torch.tensor([char_to_index[char] for char in text[i:i+seq_len]]).view(1, -1).to(device)
        targets = torch.tensor([char_to_index[char] for char in text[i+1:i+seq_len+1]]).view(1, -1).to(device)

        optimizer.zero_grad()
        hidden = model.init_hidden(1)

        loss = 0
        for j in range(seq_len):
            output, hidden = model(inputs[:,j].unsqueeze(0), hidden)
            loss += criterion(output.view(1, -1), targets[:,j])

        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

# Example usage for text generation
start_char = 'L'
num_predict = 100

with torch.no_grad():
    input_char = start_char
    hidden = model.init_hidden(1)

    for _ in range(num_predict):
        input_tensor = torch.tensor([char_to_index[input_char]]).to(device)
        output, hidden = model(input_tensor.unsqueeze(0), hidden)
        predicted_index = output.argmax().item()
        predicted_char = index_to_char[predicted_index]
        print(predicted_char, end='')
        input_char = predicted_char


Epoch [0/1000], Loss: 391.0338
Epoch [100/1000], Loss: 28.4291
Epoch [200/1000], Loss: 12.1416
Epoch [300/1000], Loss: 7.6467
Epoch [400/1000], Loss: 5.5639
Epoch [500/1000], Loss: 4.3399
Epoch [600/1000], Loss: 3.5314
Epoch [700/1000], Loss: 2.9532
Epoch [800/1000], Loss: 2.5160
Epoch [900/1000], Loss: 2.1725
orem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore e